In [ ]:
import train
import numpy as np
import api.capital_features_api as cf
import pandas as pd
import mplfinance as mpf
from datetime import datetime
import joblib
import matplotlib.pyplot as plt
from joblib import load
import tf_util as tu
import os

In [ ]:
train_ud_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1837497)
validation_ud_data = cf.get_train_data(cf.docker_train_data_url, train_config_id=1837499)
Y_ud_train = train_ud_data['upSignals']
Y_ud_val = validation_ud_data['upSignals']

In [ ]:
train_ud_dataset, validation_ud_dataset, scaler_ud = train.to_dataset(train_ud_data, validation_ud_data, Y_ud_train, Y_ud_val)
weights_ud = train.to_weights(Y_ud_train)
os.makedirs('./model/v8.80.5.200.1', exist_ok=True)
joblib.dump(scaler_ud, './model/v8.80.5.200.1/spy500-200_model_up_80_ty_scaler_ud.save')
train.save_scaler(scaler_ud, './model/v8.80.5.200.1/spy500-200_model_up_80_ty_scaler_ud.json')

In [ ]:
model_ud, callbacks_ud = train.get_model(5, 80, 'spy500-200_model_ud_80_ty')

In [ ]:
# model_ud.load_weights('./epoch/spy500-200_model_ud_80_ty/epoch_432.ckpt')
# scaler_ud = joblib.load('./model/v8.80.5.200.1/spy500-200_model_up_80_ty_scaler_ud.save')

In [ ]:
# initial_epoch=tu.find_last_epoch_txt_log('./log/epoch.log') + 1
his_ud = model_ud.fit(train_ud_dataset, epochs=1000, validation_data=validation_ud_dataset, verbose=1, callbacks=callbacks_ud, class_weight=weights_ud)
model_ud.save('./model/v8.80.5.200.1/spy500-200_model_ud_80_ty.h5')
model_ud.save('./model/v8.80.5.200.1/spy500-200_model_ud_80_ty', save_format='tf')

In [ ]:
plt.plot(his_ud.history['accuracy'], label='accuracy')
plt.plot(his_ud.history['val_accuracy'], label='val_accuracy')
plt.plot(his_ud.history['loss'], label='loss')
plt.plot(his_ud.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
prediction_data = cf.get_prediction_data(cf.docker_prediction_data_url, train_config_id=1837497, start="2021-07-11", end=datetime.now().strftime("%Y-%m-%d"), code='SPY')
kline = cf.get_kline_histroy(cf.docker_kline_history_url, code="SPY", start="2021-07-11", end=datetime.now().strftime("%Y-%m-%d"))
df = cf.his_to_df(kline)

In [ ]:
X_prediction_scaled = train.to_prediction_scaled(prediction_data, scaler_ud)
Y_ud_prediction = model_ud.predict(X_prediction_scaled)

In [ ]:
predictions_ud_series = pd.Series(Y_ud_prediction.flatten(), index=df.index)
buy_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
buy_signals[predictions_ud_series > 0.99] = df['low'][predictions_ud_series > 0.99] * 0.99
buy_markers = mpf.make_addplot(buy_signals, type='scatter', markersize=50, marker='^', color='g')

sell_signals = np.full(df.shape[0], np.nan)  # 含 NaN 的数组，与 df 的长度一致
sell_signals[predictions_ud_series < 0.01] = df['high'][predictions_ud_series < 0.01] * 1.01
sell_markers = mpf.make_addplot(sell_signals, type='scatter', markersize=50, marker='o', color='r')

mpf.plot(df, type='candle', style='charles', addplot=[buy_markers, sell_markers], volume=True, figsize=(24, 16))